In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class MeanImputer(BaseEstimator, TransformerMixin):
    def __init__(self, feature_means=None):
        self.feature_means = feature_means

    def fit(self, X, y=None):
        self.feature_means = np.nanmean(X, axis=0).tolist()
        return self

    def transform(self, X):
        X = np.asarray(X)
        return np.where(np.isnan(X), self.feature_means, X)

In [2]:
imputer = MeanImputer()
imputer.fit([[1, np.nan], [3, 4]])
params = imputer.get_params()

In [3]:
{
  'feature_means': [2.0, 4.0]
}

{'feature_means': [2.0, 4.0]}

In [4]:
params

{'feature_means': [2.0, 4.0]}

In [5]:
import json
json_string = json.dumps(imputer.get_params())

In [6]:
params = json.loads(json_string)
restored = MeanImputer(**params)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
import numpy as np
import json

# Dataset and simulated missing values
X, y = load_iris(return_X_y=True)
X[::10] = np.nan  # simulate missing data

pipe = Pipeline([
    ('imputer', MeanImputer()),
    ('clf', LogisticRegression(max_iter=500))
])

grid = GridSearchCV(pipe, param_grid={
    'clf__C': [0.1, 1.0, 10.0]
})

grid.fit(X, y)
best = grid.best_estimator_

# Serialize best estimator's parameters
# Extract only the JSON-safe parts of the params
def to_json_safe_params(params):
    def simplify(v):
        if isinstance(v, BaseEstimator):
            return v.get_params()
        return v
    return {k: simplify(v) for k, v in params.items()}

safe_params = to_json_safe_params(best.get_params(deep=True))
params_json = json.dumps(safe_params, indent=2)
print(params_json)

# Deserialize manually (for demonstration)
params_dict = json.loads(params_json)
reconstructed = Pipeline([
    ('imputer', MeanImputer(**params_dict['imputer'])),
    ('clf', LogisticRegression(max_iter=500, C=params_dict['clf__C']))
])

TypeError: Object of type MeanImputer is not JSON serializable

In [ ]:
best.get_params(deep=True)

{'memory': None,
 'steps': [('imputer',
   MeanImputer(feature_means=[5.83777777777778, 3.0548148148148155,
                              3.751851851851853, 1.1940740740740745])),
  ('clf', LogisticRegression(C=10.0, max_iter=500))],
 'transform_input': None,
 'verbose': False,
 'imputer': MeanImputer(feature_means=[5.83777777777778, 3.0548148148148155,
                            3.751851851851853, 1.1940740740740745]),
 'clf': LogisticRegression(C=10.0, max_iter=500),
 'imputer__feature_means': [5.83777777777778,
  3.0548148148148155,
  3.751851851851853,
  1.1940740740740745],
 'clf__C': 10.0,
 'clf__class_weight': None,
 'clf__dual': False,
 'clf__fit_intercept': True,
 'clf__intercept_scaling': 1,
 'clf__l1_ratio': None,
 'clf__max_iter': 500,
 'clf__multi_class': 'deprecated',
 'clf__n_jobs': None,
 'clf__penalty': 'l2',
 'clf__random_state': None,
 'clf__solver': 'lbfgs',
 'clf__tol': 0.0001,
 'clf__verbose': 0,
 'clf__warm_start': False}